In [2]:
from langchain_community.llms import Ollama
from langchain_community.utilities import SQLDatabase
from langchain_openai import ChatOpenAI
import os
from getpass import getpass

In [3]:
os.environ["OPENAI_API_KEY"] = "<<put your key here>>"

In [4]:
# llm = Ollama(model="phi3")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [5]:
db = SQLDatabase.from_uri("sqlite:///olist.db")

In [6]:
print(db.dialect)

sqlite


In [7]:
print(db.get_usable_table_names())

['olist_customers_dataset', 'olist_geolocation_dataset', 'olist_order_items_dataset', 'olist_order_payments_dataset', 'olist_order_reviews_dataset', 'olist_orders_dataset', 'olist_products_dataset', 'olist_sellers_dataset']


In [9]:
db.run("SELECT COUNT(*) from olist_customers_dataset")

'[(99441,)]'

In [10]:
from langchain.chains import create_sql_query_chain
chain = create_sql_query_chain(llm,db)
response = chain.invoke({"question":"How many customers are there"})
response

'SELECT COUNT("customer_id") AS total_customers\nFROM olist_customers_dataset'

In [11]:
db.run(response)

'[(99441,)]'

In [24]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [16]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
# chain = write_query | execute_query
# chain.invoke({"question": "How many employees are there"})

In [18]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many products are there"})

'There are a total of 32,951 products in the olist_products_dataset.'

In [19]:
from langchain_community.agent_toolkits import create_sql_agent

In [20]:
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [23]:
agent_executor.invoke(
    {
        "input": "List the product with highest count of orders."
    }
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`


olist_customers_dataset, olist_geolocation_dataset, olist_order_items_dataset, olist_order_payments_dataset, olist_order_reviews_dataset, olist_orders_dataset, olist_products_dataset, olist_sellers_dataset
Invoking: `sql_db_schema` with `{'table_names': 'olist_order_items_dataset, olist_products_dataset'}`



CREATE TABLE olist_order_items_dataset (
	order_id TEXT, 
	order_item_id INTEGER, 
	product_id TEXT, 
	seller_id TEXT, 
	shipping_limit_date TEXT, 
	price REAL, 
	freight_value REAL
)

/*
3 rows from olist_order_items_dataset table:
order_id	order_item_id	product_id	seller_id	shipping_limit_date	price	freight_value
00010242fe8c5a6d1ba2dd792cb16214	1	4244733e06e7ecb4970a6e2683c13e61	48436dade18ac8b2bce089ec2a041202	2017-09-19 09:45:35	58.9	13.29
00018f77f2f0320c557190d7a144bdd3	1	e5f2d52b802189ee658865ca93d83a8f	dd7ddc04e1b6c2c614352b383efe2d36	2017-05-03 11:05:13	239.9	19.93
00

{'input': 'List the product with highest count of orders.',
 'output': "The product with the highest count of orders is the one with the product_id 'aca2eb7d00ea1a7b8ebd4e68314663af' with a total of 527 orders."}